This notebook is used to determine which strategy is the best for any particular asset.

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd

df = pd.read_csv("../data/processed/starting_portfolio.csv")

In [3]:
import yfinance as yf

start_date = "2024-03-01"
end_date = "2025-02-28"

portfolio = {}
for asset, weight in list(map(list, df.values)):
    portfolio[asset] = {
        "data": yf.Ticker(asset).history(start=start_date, end=end_date, actions=False),
        "weight": weight,
        "strategy": None,
        "return": float("-inf"),
    }

## Testing with one asset and one strategy

In [4]:
from backtesting import Backtest

from strategies.scalping import Scalping

data = portfolio["AVAX-USD"]["data"]
data.index = data.index.values.astype("datetime64[D]")
bt = Backtest(
    data, Scalping, cash=1000000, commission=0.002, exclusive_orders=True
)
stats = bt.run()
bt.plot()

return_pct = stats["Return [%]"]
if return_pct >= portfolio["AVAX-USD"]["return"]:
    portfolio["AVAX-USD"]["strategy"] = bt._strategy.__name__
    portfolio["AVAX-USD"]["return"] = return_pct

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_stats.py:155: RuntimeWarning: divide by zero encountered in log
  equity_log_returns = np.log(equity[1:] / equity[:-1])


In [5]:
stats

Start                     2024-03-01 00:00:00
End                       2025-02-27 00:00:00
Duration                    363 days 00:00:00
Exposure Time [%]                    28.02198
Equity Final [$]                          0.0
Equity Peak [$]                 1160717.05056
Commissions [$]                    6128.50606
Return [%]                             -100.0
Buy & Hold Return [%]                -58.8204
Return (Ann.) [%]                           0
Volatility (Ann.) [%]               288.96146
CAGR [%]                               -100.0
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Alpha [%]                                 NaN
Beta                                      NaN
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                   -22.80263
Max. Drawdown Duration      174 days 00:00:00
Avg. Drawdown Duration       37 days 00:00:00
# Trades                          

## Testing with all assets and one strategy

In [6]:
from backtesting import Backtest

from strategies.scalping import Scalping


for asset in portfolio:
    data = portfolio[asset]["data"]
    data.index = data.index.values.astype("datetime64[D]")
    bt = Backtest(
        data, Scalping, cash=1000000, commission=0.002, exclusive_orders=True
    )
    stats = bt.run()
    # bt.plot()
    return_pct = stats["Return [%]"]
    if return_pct >= portfolio[asset]["return"]:
        portfolio[asset]["strategy"] = bt._strategy.__name__
        portfolio[asset]["return"] = return_pct

/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_stats.py:155: RuntimeWarning: divide by zero encountered in log
  equity_log_returns = np.log(equity[1:] / equity[:-1])
/Users/valerietan/.pyenv/versions/cryptoto/lib/python3.12/site-packages/backtesting/_stats.py:155: RuntimeWarning: divide by zero encountered in log
  equity_log_returns = np.log(equity[1:] / equity[:-1])


In [7]:
all_assets_one_strategy = pd.DataFrame(
    [
        [
            asset,
            portfolio[asset]["strategy"],
            portfolio[asset]["return"]
        ]
        for asset in portfolio
    ],
    columns=["asset", "strategy", "return"]
)

In [8]:
all_assets_one_strategy

,asset,strategy,return
0,CZR,Scalping,-12.067173
1,INTC,Scalping,19.298676
2,MHK,Scalping,-8.178805
3,BLDR,Scalping,26.837742
4,URI,Scalping,18.386381
5,ON,Scalping,0.000000
6,NCLH,Scalping,-37.950438
7,ALB,Scalping,18.432952
8,VST,Scalping,9.985069
9,AVAX-USD,Scalping,-100.000000


## All assets and all strategies

In [ ]:
from backtesting import Backtest

from strategies.larry_williams_price_action import LarryWilliamsPriceAction
from strategies.macd_bollinger_bands_mean_reversion import MACDBollingerBandsMeanReversion
from strategies.michael_harris_price_action import MichaelHarrisPriceAction
from strategies.rsi_divergence import RSIDivergence
from strategies.scalping import Scalping
from strategies.volume_spike_reversal import VolumeSpikeReversal
from strategies.bollinger_bands_breakout import BollingerBandsBreakout

strategies = [
    BollingerBandsBreakout,
    LarryWilliamsPriceAction,
    MACDBollingerBandsMeanReversion,
    MichaelHarrisPriceAction,
    RSIDivergence,
    Scalping,
    VolumeSpikeReversal,
]

for asset in portfolio:
    for strategy in strategies:
        data = portfolio[asset]["data"]
        data.index = data.index.values.astype("datetime64[D]")
        bt = Backtest(
            data, strategy, cash=1000000, commission=0.002, exclusive_orders=True
        )
        stats = bt.run()
        return_pct = stats["Return [%]"]
        if return_pct >= portfolio[asset]["return"]:
            portfolio[asset]["strategy"] = bt._strategy.__name__
            portfolio[asset]["return"] = return_pct

In [ ]:
results = pd.DataFrame(
    [
        [
            asset,
            portfolio[asset]["weight"],
            portfolio[asset]["strategy"],
        ]
        for asset in portfolio
    ],
    columns=["asset", "weight", "strategy"]
)

results.to_csv("../data/misc/example_backtesting.csv", index=False)